In [1]:
# import library transformers
from transformers import BertTokenizer, BertForSequenceClassification
# import matplotlib
import matplotlib.pyplot as plt
# import sklearn
from sklearn.metrics import confusion_matrix
# import seaborn
import seaborn as sns

# import torch
import torch

# import safetensors
from safetensors.torch import load_file
# import pandas
import pandas as pd
import os
from tqdm import tqdm 

# pilih versi model
versi_model = 13
# inmport dataset
df = pd.read_csv("dataset/barokah-1.csv", sep="|")

c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Encode labels to integers
df['label'] = df['answer'].astype('category').cat.codes
# Create a dictionary to map labels to their integer values
label_dict = dict(enumerate(df['answer'].astype('category').cat.categories))

# Load the model and tokenizer
tokenizer = BertTokenizer.from_pretrained("./models/model v"+str(versi_model))
# Load the model
model = BertForSequenceClassification.from_pretrained(
    "./models/model v"+str(versi_model))

# create function to get answer
def get_answer(question):
    inputs = tokenizer(question, return_tensors="pt")
    outputs = model(**inputs)
    answer = torch.argmax(outputs.logits).item()
    return label_dict[answer]
    # return outputs

In [3]:
df.head()

,question,answer,label
0,pembayaran online,Pembayaran online saat ini dapat dilakukan mel...,4767
1,pengiriman barang,Pengiriman barang kini dapat dilacak secara re...,4873
2,sistem keamanan rumah,Sistem keamanan rumah modern menggunakan tekno...,6085
3,e-learning,E-learning memungkinkan siswa untuk mengakses ...,1957
4,perpustakaan digital,Perpustakaan digital menyediakan akses ke ribu...,5087


In [4]:
# List untuk menampung hasil prediksi
predicted_labels = []

# Iterasi pada setiap baris data
for index, row in tqdm(df.iterrows(), total=len(df)):
    text = row['question']
    predicted_label = get_answer(text)
    predicted_labels.append(predicted_label)

# Menambahkan hasil prediksi ke dataframe baru
df['Predicted_Label'] = predicted_labels

100%|██████████| 8000/8000 [04:24<00:00, 30.24it/s]


In [5]:
df.head()

,question,answer,label,Predicted_Label
0,pembayaran online,Pembayaran online saat ini dapat dilakukan mel...,4767,Pembayaran online saat ini dapat dilakukan mel...
1,pengiriman barang,Pengiriman barang kini dapat dilacak secara re...,4873,Pengiriman barang kini dapat dilacak secara re...
2,sistem keamanan rumah,Sistem keamanan rumah modern menggunakan tekno...,6085,Sistem keamanan rumah modern menggunakan tekno...
3,e-learning,E-learning memungkinkan siswa untuk mengakses ...,1957,E-learning memungkinkan siswa untuk mengakses ...
4,perpustakaan digital,Perpustakaan digital menyediakan akses ke ribu...,5087,Perpustakaan digital menyediakan akses ke ribu...


In [6]:
# Hitung akurasi
correct_predictions = (df['Predicted_Label'] == df['answer']).sum()
total_predictions = len(df)
accuracy = correct_predictions / total_predictions * 100
print(f'Correct predictions: {correct_predictions}/{total_predictions}')
print(f'Accuracy: {accuracy:.2f}%')

Correct predictions: 7234/8000
Accuracy: 90.42%
